In [4]:
import numpy as np
import matplotlib as plt
import pandas as pd
import xml.etree.ElementTree as ET
import io
#!pip install -U -q PyDrive
import os
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

**Working import of a sample xml file from colab/drive**

In [5]:
# ##Note: if you are updating a file, re-run this cell to update the image of drive

# # 1. Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# # 2.  mount the google drive into colab. Now you can use terminal commmands like
# #     !ls, !pwd, etc to navigate to your files. You should get a message like 
# #     'Drive already mounted' or, it will ask you to authenticate with google.
# from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)



 **Importing annotation file**

**Extracting top and bottom text annotation, and separate the XML **

In [6]:
mypath = 'PowerAnnotations_V1.0/AnnotatedThreads/non train/'

#run this for train in case you need it
#mypath = 'PowerAnnotations_V1.0/AnnotatedThreads/train/'


In [7]:
# get all files in folder.

from os import listdir
from os.path import isfile, join
filesInDir = [f for f in listdir(mypath) if (isfile(join(mypath, f)) and f[-4:] == '.xml')]
filesInDir

['thread99.MN.infoadded.xml',
 'thread104.MN.infoadded.xml',
 'thread86.MN.infoadded.xml',
 'thread92.MN.infoadded.xml',
 'thread89.MN.infoadded.xml',
 'thread96.MN.infoadded.xml',
 'thread100.MN.infoadded.xml',
 'thread82.MN.infoadded.xml',
 'thread105.MN.infoadded.xml',
 'thread98.MN.infoadded.xml',
 'thread93.MN.infoadded.xml',
 'thread88.MN.infoadded.xml',
 'thread65.MN.infoadded.xml',
 'thread83.MN.infoadded.xml',
 'thread101.MN.infoadded.xml',
 'thread90.MN.infoadded.xml',
 'thread84.MN.infoadded.xml',
 'thread106.MN.infoadded.xml',
 'thread66.MN.infoadded.xml',
 'thread102.MN.infoadded.xml',
 'thread94.MN.infoadded.xml',
 'thread72.MN.infoadded.xml',
 'thread109.MN.infoadded.xml',
 'thread91.MN.infoadded.xml',
 'thread68.MN.infoadded.xml',
 'thread85.MN.infoadded.xml',
 'thread103.MN.infoadded.xml',
 'thread81.MN.infoadded.xml',
 'thread67.MN.infoadded.xml',
 'thread108.MN.infoadded.xml',
 'thread95.MN.infoadded.xml']

In [48]:
data = {}
for i, f in enumerate(filesInDir):
    file = open(mypath + f, 'r' , encoding="ISO-8859-1")
    data[f] = file.read()


In [49]:
def writeFile(name, data):
    fw = open(name, "w+")
    fw.write(data)
    fw.close()
    return

In [51]:
#read each file in directory, then extract top & bottom annotations, amd XML. 
# Write xml to a separate dir.
import re
import os.path

save_path = '/Users/t_sura/Desktop/School/NLP/project/NLP_Final_Project/Clean XML files'

#name_of_file = raw_input("What is the name of the file: ")

#
df = pd.DataFrame(columns = ['filename', 'top_annotation', 'bottom_annotation'])
  
rownum = 0
topAnno = []
bottomAnno = []
xmlfilename = []
columnOfXML = []
for k, v in data.items():
    searchObj = re.search('<custom>(.|\n)*?<\/custom>', v)
    #searchObj = re.search('<xml>(.|\n)*?<\/xml>', v)
    separatedXML = re.search('<custom>(.|\n)*?<\/custom>', v).group(0)
    # --- run this line below for train data ---
    separatedXML = re.search('<xml>(.|\n)*?<\/xml>', v).group(0)
    # ---
    columnOfXML.append(separatedXML)
    (startOfXML, endOfXML) = searchObj.span()
    topAnno.append(v[:startOfXML])
    bottomAnno.append(v[endOfXML:])
    xmlfilename.append('clean_xml_' + k)
    
    name = 'clean_xml_' + k
    completeName = os.path.join(save_path, name)  
    writeFile(completeName, separatedXML)

df['filename'] = pd.Series(xmlfilename)
df['top_annotation'] = pd.Series(topAnno)
df['bottom_annotation'] = pd.Series(bottomAnno)
df['XML'] = pd.Series(columnOfXML)

AttributeError: 'NoneType' object has no attribute 'group'

In [9]:
#df.to_csv('separat\ed_Data.csv')
df

,filename,top_annotation,bottom_annotation


In [52]:
#df.to_csv('separated_Data.csv')

### Cleaning data further

##### 1. MTags for example M3.6. and M3.10
##### 2. Removed [....]

In [44]:
def extract_message(data):
    
    #parser1 = ET.XMLParser(encoding="ISO-8859-1")
    tree = ET.ElementTree(ET.fromstring(data))
    root = tree.getroot()
    
    message = []
    for each in root.iter(tag='row'):
        message.append(each)
    
    text_message = []
    
    for each in message:
        text_message.append(ET.tostring(each))
    #print(text_message)
    return text_message

In [45]:
path = "Clean XML files/"
name = 'clean_xml_thread98.MN.infoadded.xml'
f = open(path  + name)
data = f.read()
f.close()


def removeMTagsAndBracketComments(data):
# removing mtags
    sansMTags1 = re.sub('M[\d]?\.[\d]+\.', '' , data)
    sansMTags2 = re.sub('M[\d]?\.[\d]+', '', sansMTags1)
    sansBracketComments = re.sub('\[.+\]', '' , sansMTags2)
    return sansBracketComments


sansBracketComments = removeMTagsAndBracketComments(data)
#write
name = 'test_sansMtag2.txt'
writeFile(name, sansBracketComments)

#sansBracketComments

extract_message(data)


ParseError: not well-formed (invalid token): line 158, column 25 (<string>)

In [23]:
#sansMessageEndTag

In [48]:
sObj = re.search('<content>(.|\n)*?\<\/content>', sansBracketComments)
sObj.group(0)

"<content>\n\n Andy,\n\n\n\n\n\n Attached are some ideas for possible charge structures for EnronOnline.\n\n I am recommending something which will probably be surprising, given our conversation.\n\n Let's discuss when you have a moment.\n\n\n\n\n\n Dave\n\n\n\n\n\n\n\n Recommendation\n\n\n\n\n\n a) For new commodity areas, continue to charge a set up fee in accordance with our previously agreed schedule.\n\n (e.g. $350,000 for new Market Area)\n\n\n\nSflink1.8\n\n\n\n b) Charge a per-volume maintenance fee which is comparable to industry brokerage fees,\n\n with a minimum charge equivalent to $4,000 per Product * Total number of Products).\n\n\n\nSflink1.10\n\n\n\n This method results in a total charge of approx. $46.5 million pa.\n\n (providing coverage for existing charges, some growth and London's charges)\n\n\n\n\n\n This method is recommended because it balances a fee to reflect real expenditure of effort on behalf on Enron Online staff (the per Product minimum)\n\n with a struct

### Extracting body of message

In [49]:
def getBodyOfMessage(sansBracketComments):
    emailBodys = []
    i = re.finditer('<content>(.|\n)*?\<\/content>', sansBracketComments)
    for each in i:
        emailBodys.append(each.group(0))

    body = ''
    for text in emailBodys:
        body = body + text


    #removing the <tag>
    body = re.sub('<content>', '' , body)
    body = re.sub('</content>', '' , body)
    body = re.sub('<custom>', '' , body)
    body= re.sub('</custom>', '' , body)

    body= re.sub('Sflink[\d]+.[\d]+', '' , body)
    body= re.sub('Blink[\d]+.[\d]+', '' , body)
    body= re.sub('Flink[\d]+.[\d]+', '' , body)

    return body

name = 'sampleBody.txt'
writeFile(name, body)



In [17]:
#Fetching the 'from' tags
import xml.etree.ElementTree as ET
tree = ET.ElementTree(ET.fromstring(data))
root = tree.getroot()

for something in root.iter(tag='from'):
    print(something.get('name'))

David Forster
David Forster
David Forster
David Forster
David Forster
David Forster


In [18]:
for something in root.iter(tag='message_id'):
    print(something.text)

106860
2000020
106859
2000021
106858


[<Element 'row' at 0x117aae728>,
 <Element 'row' at 0x117ad0548>,
 <Element 'row' at 0x117ad0868>,
 <Element 'row' at 0x117ad0cc8>,
 <Element 'row' at 0x117ad0f98>]